In [25]:
#!pip install nlpaug
#!pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/60/90/81ac364ef94209c100e12579629dc92bf7a709a84af32f8c551b02c07e94/nltk-3.9.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.5 MB 6.3 MB/s eta 0:00:01
   ------------------------------ --------- 1.1/1.5 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 10.7 MB/s eta 0:00:00


In [22]:
# Importing useful dependencies
import boto3
import re
import random
from typing import List
import nltk
from nltk.corpus import wordnet
import torch
import open_clip
import numpy as np
# Set a seed for reproducibility
SEED = 10721
random.seed(SEED) 
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [2]:
# Setup S3 client for MinIO (MinIO implements Amazon S3 API)
s3 = boto3.client(
    "s3",
    endpoint_url="http://127.0.0.1:9000", # MinIO API endpoint
    aws_access_key_id="minioadmin", # User name
    aws_secret_access_key="minioadmin", # Password
)


In [3]:
# We create a new Bucket in Min-IO to store our augmented data

# List existing buckets
buckets = [b["Name"] for b in s3.list_buckets()["Buckets"]]

# Function that given a name, creates a bucket
def createBucket(name, list_buckets):
    if name in list_buckets:
        print(f"Bucket '{name}' already exists!")
    else:
        s3.create_bucket(Bucket=name)
        print(f"Created bucket: {name}")

# Create a bucket named landing_zone
createBucket("training-data-construction-zone", buckets)
# Sub-bucket: Baseline Training Data
s3.put_object(Bucket="training-data-construction-zone", Key="texts/")

Bucket 'training-data-construction-zone' already exists!


{'ResponseMetadata': {'RequestId': '187A0F2DA625D69D',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-checksum-crc32': 'AAAAAA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '187A0F2DA625D69D',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '2109',
   'x-ratelimit-remaining': '2109',
   'x-xss-protection': '1; mode=block',
   'date': 'Fri, 21 Nov 2025 15:28:12 GMT'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ChecksumCRC32': 'AAAAAA==',
 'ChecksumType': 'FULL_OBJECT'}

In [4]:
def get_text(bucket, key):
    resp = s3.get_object(Bucket=bucket, Key=key)
    body = resp["Body"].read()
    text = body.decode("utf-8")
    return text

In [5]:
# Just in case our device has gpu
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model
model, _, _ = open_clip.create_model_and_transforms("hf-hub:laion/CLIP-ViT-L-14-laion2B-s32B-b82K")
tokenizer = open_clip.get_tokenizer("hf-hub:laion/CLIP-ViT-L-14-laion2B-s32B-b82K") # Tokenizer for texts
model.to(device)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1024,), eps=1e-05, elementwi

In [6]:
@torch.no_grad()
# The next function returns the embedding of the given text
def embed_text(model, tokenizer, texts: str):
    tokens = tokenizer([texts]).to(device) # tokenized batch
    feats = model.encode_text(tokens)
    feats = feats / feats.norm(dim=-1, keepdim=True) # normalize
    return feats.cpu().numpy()[0]

In [26]:
#  Very simple tokenizer for English-like text.
def simple_tokenize(text: str) -> List[str]:
    return re.findall(r"\w+|[^\w\s]", text, re.UNICODE)

#  Simple detokenizer matching simple_tokenize.
def detokenize(tokens: List[str]) -> str:
    """
    Simple detokenizer matching simple_tokenize.
    """
    text = " ".join(tokens)
    text = re.sub(r"\s+([.,!?;:])", r"\1", text)
    return text

In [25]:
# Delete each token with probability p.
def random_deletion(tokens: List[str], p: float = 0.1) -> List[str]:
    if len(tokens) == 1:
        return tokens

    kept = [t for t in tokens if random.random() > p]
    if not kept:
        kept.append(random.choice(tokens))
    return kept

# Randomly swap a small portion of tokens.
def random_swap(tokens: List[str], ratio: float = 0.05) -> list[str]:
    n = len(tokens)
    if n < 2:
        return tokens

    n_swaps = max(1, int(ratio * n))

    for _ in range(n_swaps):
        i, j = random.sample(range(n), 2)
        tokens[i], tokens[j] = tokens[j], tokens[i]
    return tokens

In [39]:
ALPHABET = "abcdefghijklmnopqrstuvwxyz"
# Introduce a simple spelling error in a single word
def corrupt_word(word: str) -> str:
    if len(word) == 0:
        return word

    op = random.choice(["delete", "substitute", "duplicate"])

    if op == "delete" and len(word) > 1:
        pos = random.randrange(len(word))
        return word[:pos] + word[pos+1:]

    if op == "substitute":
        pos = random.randrange(len(word))
        new_char = random.choice(ALPHABET)
        return word[:pos] + new_char + word[pos+1:]

    if op == "duplicate":
        pos = random.randrange(len(word))
        return word[:pos] + word[pos] + word[pos:]

    return word

# For each alphabetical token, apply a spelling error with probability p.
def random_spelling_error(tokens: List[str], p: float = 0.1) -> List[str]:
    new_tokens = []
    for t in tokens:
        if t.isalpha() and random.random() < p:
            new_tokens.append(corrupt_word(t))
        else:
            new_tokens.append(t)
    return new_tokens

In [37]:
# Collect synonyms from WordNet (very naive).
def get_synonyms(word: str) -> List[str]:
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            lemma_name = lemma.name().replace("_", " ")
            if lemma_name.lower() != word.lower():
                synonyms.add(lemma_name)
    return list(synonyms)

# Randomly choose a small portion of tokens and replace them with synonyms.
def random_synonym_replacement(tokens: List[str], ratio: float = 0.05) -> List[str]:
    n = len(tokens)
    n_replacements = max(1, int(ratio * n))
    candidate_indices = [
        i for i, t in enumerate(tokens)
        if t.isalpha() and len(t) > 2  # skip punctuation & very short tokens
    ]
    if not candidate_indices:
        return tokens

    indices = random.sample(candidate_indices, n_replacements)

    for idx in indices:
        word = tokens[idx]
        syns = get_synonyms(word)
        if syns:
            tokens[idx] = random.choice(syns)
    return tokens

In [41]:
def augment(method,text,ratio = 0.1):
    tokens = simple_tokenize(text)
    if (method == "swap_spelling_aug"):
        # We apply two augmentations, so we halve the ratio for each one.
        ratio = ratio / 2
        tokens = random_spelling_error(tokens,ratio)
        tokens = random_swap(tokens,ratio)

    elif (method == "delete_aug"):
        tokens = random_deletion(tokens, ratio)

    elif (method == "swap_sym_word_aug"):
        # We apply two augmentations, so we halve the ratio for each one.
        ratio = ratio / 2
        tokens = random_synonym_replacement(tokens,ratio)
        tokens = random_swap(tokens,ratio) 

    else:
        raise ValueError(f"Unknown augmentation method: {method}")

    return detokenize(tokens)

In [48]:
#Generate an augmented version of original text and save the reuslt to S3
def generate_and_save_augmented(
    suffix: str,
    body: str,
    embedded_orig: np.ndarray,
    model_text,
    tokenizer,
    dest_bucket: str,
    dest_prefix: str,
    new_key_infix: str,
    max_try: int = 10,
    min_sim_accept: float = 0.9,
    max_sim_accept: float = 0.99,
):
    best_text = None
    best_sim = -1.0

    for attempt in range(max_try):
        # 1. Generate augmented text
        aug_text = augment(suffix,body)
        
        # 2. Compute cosine similarity with the original text embedding
        emb_aug = embed_text(model_text, tokenizer, aug_text)
        sim = float(np.dot(embedded_orig, emb_aug))
        

        # Track the best candidate regardless of whether it is in the range
        if sim > best_sim and sim < max_sim_accept:
            best_sim = sim
            best_text = aug_text

        elif sim < abs(best_sim) and  sim > max_sim_accept:
            best_sim = sim
            best_text = aug_text

        # If similarity is within the acceptable range, save and return early
        if min_sim_accept <= sim <= max_sim_accept:
            augment_key = f"{dest_prefix}{new_key_infix}_{suffix}.txt"
            s3.put_object(
                Bucket=dest_bucket,
                Key=augment_key,
                Body=aug_text.encode("utf-8"),
                ContentType="text/plain",
            )
            print(f"[{suffix}] attempt {attempt + 1}, sim={sim:.4f}")
            return best_sim

    # If no candidate falls into [min_sim_accept, max_sim_accept],
    # use the best candidate found as a fallback.
    if best_text is not None:
        augment_key = f"{dest_prefix}{new_key_infix}_{suffix}.txt"
        print(f"[{suffix}] use best_sim={best_sim:.4f} as fallback")
        s3.put_object(
            Bucket=dest_bucket,
            Key=augment_key,
            Body=best_text.encode("utf-8"),
            ContentType="text/plain",
        )

    return best_sim

In [31]:
def text_augmentation(src_bucket, dest_bucket, model_text, tokenizer, dest_prefix="texts/"):
    paginator = s3.get_paginator("list_objects_v2") # It returns objects in pages and not all at once.
    for page in paginator.paginate(Bucket=src_bucket, Prefix="baseline-training-data/"):

        for obj in page.get("Contents", []):
            key = obj["Key"]

            if "text" in key:
                body = get_text(src_bucket, key)
                embedded = embed_text(model_text,tokenizer,body)
                new_key_infix = key.split("/")[1].split(".")[0]
                new_key = dest_prefix + new_key_infix + ".txt"
                copy_source_text = {"Bucket": src_bucket, "Key": key}
                s3.copy_object(Bucket=dest_bucket, Key=new_key, CopySource=copy_source_text)
                # 1) word-level insert spelling error
                generate_and_save_augmented(
                    suffix="swap_spelling_aug",
                    body=body,
                    embedded_orig=embedded,
                    model_text=model_text,
                    tokenizer=tokenizer,
                    dest_bucket=dest_bucket,
                    dest_prefix=dest_prefix,
                    new_key_infix=new_key_infix,
                    max_try=10,
                    min_sim_accept=0.9,
                    max_sim_accept=0.99,
                )
    
                # 2) Word-level delete
                generate_and_save_augmented(
                    suffix="delete_aug",
                    body=body,
                    embedded_orig=embedded,
                    model_text=model_text,
                    tokenizer=tokenizer,
                    dest_bucket=dest_bucket,
                    dest_prefix=dest_prefix,
                    new_key_infix=new_key_infix,
                    max_try=10,
                    min_sim_accept=0.9,
                    max_sim_accept=0.99,
                )
    
                # 3) Word-level swap + synonym
                generate_and_save_augmented(
                    suffix="swap_sym_word_aug",
                    body=body,
                    embedded_orig=embedded,
                    model_text=model_text,
                    tokenizer=tokenizer,
                    dest_bucket=dest_bucket,
                    dest_prefix=dest_prefix,
                    new_key_infix=new_key_infix,
                    max_try=10,
                    min_sim_accept=0.9,
                    max_sim_accept=0.99,
                )

                

In [47]:
text_augmentation(src_bucket = "training-data-construction-zone", dest_bucket = "training-data-construction-zone", model_text = model, tokenizer=tokenizer)

Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-player competition while being submerged in a unique new universe filled with over-the-top humor, wild characters, unique levels, and addictive game play. The title is aimed at players of all ages and skill sets. Through accessible and intuitive controls and game-play, Galactic Bowling allows you to jump right into the action. A single-player campaign and online play allow you to work your way up the ranks of the Galactic Bowling League! Whether you have hours to play or only a few minutes, Galactic Bowling is a fast paced and entertaining experience that will leave you wanting more! Full Single-player story campaign including 11 Characters and Environments. 2 Single-player play modes including Regular and Battle Modes. Head to Head Online Multiplayer play Modes. Super Powers, Special Balls, and Whammies. Unlockable Characters, Environments, and M

KeyboardInterrupt: 